In [43]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
from sklearn.model_selection import train_test_split
import glob
import os
from tqdm import tqdm


### Feature engineering

In [41]:
def gen_features(X):
    strain = []
    strain.append(X.mean())
    strain.append(X.std())
    strain.append(X.min())
    strain.append(X.max())
    strain.append(X.kurtosis())
    strain.append(X.skew())
    strain.append(np.quantile(X,0.95))
    strain.append(np.quantile(X,0.90))
    strain.append((X.loc[abs(X - X.mean()) < 20]).kurtosis()) # truncated kurtosis 
    strain.append(np.quantile(X,0.75) - np.quantile(X,0.25)) #iqr
    ### std_nopeak: https://stackoverflow.com/questions/51006163/pandas-how-to-detect-the-peak-points-outliers-in-a-dataframe
    df = X.copy(deep = True) #temp df
    from scipy import stats
    df_Z = df[(np.abs(stats.zscore(df)) < 2.0)] # Use z-score of 2 to remove peaks
    ix_keep = df_Z.index
    df_keep = df.loc[ix_keep] # Subset the raw dataframe with the indexes you'd like to keep
    strain.append(df_keep.std())
    ### mfcc - https://www.kaggle.com/ilu000/1-private-lb-kernel-lanl-lgbm/
    # import librosa
    # mfcc = librosa.feature.mfcc(X.values)
    # strain.append(mfcc.mean(axis=1))
    ### power spectrum
    from scipy.signal import find_peaks, periodogram
    fs = 1.0
    f, Pxx_spec = periodogram(X, fs, 'flattop')
    strain.append(sum(Pxx_spec)/len(Pxx_spec)) #mean!
    # Peaks and rolling std
    strain.append(len(find_peaks(X, height=100)[0])) # peak count
    strain.append(np.quantile(X.rolling(50).std().dropna(), 0.2)) # rolling stdev
    return pd.Series(strain)

### Load in training data and generate features for each 150k chunk

In [59]:
train = pd.read_csv('LANL-Earthquake-Prediction/train.csv', iterator=True, chunksize=150_000, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

X = pd.DataFrame()
y = pd.Series(dtype=np.float64)

i = 0
for df in train:
    ch = gen_features(df['acoustic_data'])
    if i < 5:
        df.to_csv('showcase_' + str(i) + ".csv", index=False)
    else:
        X = X.append(ch, ignore_index=True)
        y = y.append(pd.Series(df['time_to_failure'].values[-1]))
    i = i + 1



In [44]:
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,4.884113,5.101106,-98.0,104.0,33.662481,-0.024061,11.0,10.0,2.402105,4.0,3.516980,43.591004,1.0,2.652761
1,4.725767,6.588824,-154.0,181.0,98.758517,0.390561,12.0,10.0,2.527633,5.0,3.795519,49.149706,13.0,2.660674
2,4.906393,6.967397,-106.0,140.0,33.555211,0.217391,13.0,10.0,2.554792,5.0,4.039933,63.349899,6.0,2.738687
3,4.902240,6.922305,-199.0,197.0,116.548172,0.757278,12.0,10.0,2.568523,5.0,3.848283,202.262267,11.0,2.672918
4,4.908720,7.301110,-126.0,145.0,52.977905,0.064531,12.0,10.0,2.709675,5.0,3.918423,207.905019,7.0,2.677876


In [45]:
feature_names = list(X.columns)
feature_names

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

### Split data into train and test sets

In [47]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.25)

In [48]:
train_pool = Pool(train_X, train_y, feature_names=feature_names)
test_pool = Pool(test_X, test_y, feature_names=feature_names)

### Run feature selection

In [31]:
model = CatBoostRegressor(iterations=10000, loss_function='MAE')
summary = model.select_features(
    train_pool,
    eval_set=test_pool,
    features_for_select=list(range(train_pool.num_col())),
    num_features_to_select=5,
    steps=3,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Regular,
    train_final_model=True,
    logging_level='Silent',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [32]:
print('Selected features:', summary['selected_features_names'])

Selected features: ['0', '6', '8', '10', '13']


### Train model

In [49]:
train_pool = Pool(X, y)
m = CatBoostRegressor(iterations=10000, loss_function='MAE')
m.fit(X, y, silent=True)
m.best_score_

{'learn': {'MAE': 1.0088632854685071}}

### Get evaluation metrics

In [50]:
score =m.score(X, y) 
score

0.7849884875190286

In [51]:
mae = m.get_best_score().get('learn').get('MAE')
mae

1.0088632854685071

In [52]:
# initialize list of lists
data = [[score, mae]]
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Score', 'MAE'])

In [53]:
df.to_csv('EarthquakeModelMetrics.csv', index=False)

### Save model

In [54]:
m.save_model('EarthquakeModel.cbm')

### Load a single test file and predict

In [55]:
### NEW WAY FOR LOADING ONE TEST FILE (based on how loading training data above, since not shown in the github repo)
test = pd.read_csv('showcase_0.csv') # 'LANL-Earthquake-Prediction/test/seg_0a0fbb.csv'

X_test = pd.DataFrame()
y_test = pd.Series(dtype=np.float64)
ch = gen_features(test['acoustic_data'])
X_test = X_test.append(ch, ignore_index=True)
y_test = y_test.append(pd.Series(test['time_to_failure'].values[-1]))
# no y_test because no time to failure column

In [56]:
# Making prediction 
preds = m.predict(X_test) # m.predict(showcase_x)
print(preds) 
print(y_test) # print(showcase_y.average)

[9.89920682]
